# Probabilistic Programming Languages

In [2]:
from scipy import stats
import pymc3 as pm
import theano
import numpy as np
import matplotlib.pyplot as plt
import arviz as az
az.style.use("arviz-grayscale")

## Getting the Gradient

In [3]:
from jax import grad
from jax.scipy import stats

simple_grad = grad(lambda x: x**2)
simple_grad(4.0)

DeviceArray(8., dtype=float32)

In [4]:
from jax import grad
from jax.scipy import stats

def model(test_point, observed):
    z_pdf = stats.norm.logpdf(test_point, loc=0, scale=5)
    x_pdf = stats.norm.logpdf(observed, loc=test_point, scale=1)
    logpdf = z_pdf + x_pdf
    return logpdf

model_grad = grad(model)

observed, test_point = 5.0, 2.5 
logp_val = model(test_point, observed)
grad = model_grad(test_point, observed)
print(f"log_p_val: {logp_val}")
print(f"grad: {grad}")

log_p_val: -6.697315216064453
grad: 2.4000000953674316


In [5]:
with pm.Model() as model:
    z = pm.Normal("z", 0., 5.)
    x = pm.Normal("x", mu=z, sd=1., observed=observed)

func = model.logp_dlogp_function()
func.set_extra_values({})
print(func(np.array([test_point])))

[array(-6.69731498), array([2.4])]


In [8]:
def fraud_detector(obs_fraud, obs_non_fraud, fraud_prior=8, non_fraud_prior=6):
    """Conjugate beta binomial model for fraud detection"""
    
    alpha_post = fraud_prior + observed_fraud
    beta_post = fraud_prior + observed_fraud + non_fraud_prior + obs_non_fraud
    expectation = alpha_post / (alpha_post + beta_post)
    
    if expectation > .5:
        return {"suspend_card":True}

%timeit fraud_detector(2, 0)

141 ns ± 0.647 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


## Log Probability

In [21]:
observed = np.repeat(2, 2)
pdf = stats.norm(0,1).pdf(observed)
np.prod(pdf, axis=0)

0.0029150244650281948

In [24]:
observed = np.repeat(2, 1000)
pdf = stats.norm(0,1).pdf(observed)
np.prod(pdf, axis=0)

0.0

In [11]:
1.2-1

0.19999999999999996

In [12]:
pdf[0], np.prod(pdf, axis=0)

(0.05399096651318806, 0.0)

In [29]:
logpdf = stats.norm(0,1).logpdf(observed)
np.log(pdf[0]), logpdf[0], logpdf.sum()

(-2.9189385332046727, -2.9189385332046727, -2918.9385332046736)

In [28]:
np.log(pdf[0])

-2.9189385332046727

## Distribution Transforms

In [ ]:
lower, upper = -1, 2
domain = np.linspace(lower, upper, 5)
transform = np.log(domain - lower) - np.log(upper - domain)
print(f"Original domain: {domain}")
print(f"Transformed domain: {transform}")

In [ ]:
with pm.Model() as model_transform:
    x = pm.Uniform("x", -1., 2.)

In [ ]:
model_transform.vars

In [ ]:
print(model_transform.logp({"x_interval__":-2})), 
print(model_transform.logp({"x_interval__":1}))
print(model_transform.logp_nojac ({"x_interval__":-2}))
print(model_transform.logp_nojac ({"x_interval__":1}))

In [ ]:
print(model_transform.logp({"x_interval__":-2}),
        model_transform.logp_nojac({"x_interval__":-2}))
print(model_transform.logp({"x_interval__":1}),
          model_transform.logp_nojac({"x_interval__":1}))

### Example: Transforms

In [ ]:
from scipy import stats

y_observed = stats.norm(0, .01).rvs(20)

In [ ]:
with pm.Model() as model_transform:
    sd = pm.HalfNormal("sd", 5)
    y = pm.Normal("y", mu=0, sd=sd, observed=y_observed)
    trace_transform = pm.sample(chains=1, draws=100000, random_seed=0)
model.vars

In [ ]:
model_transform.vars

In [ ]:
f'Diverging: {trace_transform.get_sampler_stats("diverging").sum()}'

In [ ]:
az.plot_trace(trace_transform, combined=True, divergences="bottom", kind="rank_bars")
plt.savefig("img/model_transformed.png")

In [ ]:
with pm.Model() as model:
    sd = pm.HalfNormal("sd", 5, transform=None)
    y = pm.Normal("y", mu=0, sd=sd, observed=y_observed)
    trace_no_transform = pm.sample(chains=1, draws=100000, random_seed=42)

In [ ]:
model.vars

In [ ]:
trace_transform.get_sampler_stats("diverging").sum()

In [ ]:
az.plot_trace(trace_no_transform, combined=True, divergences="bottom", kind="rank_bars")
plt.savefig("img/model_not_transformed.png");

## Operation Graphs

In [ ]:
x=3
y=1
x*y/x + 2

In [ ]:
theano.config.compute_test_value = 'ignore'

In [ ]:
x = theano.tensor.vector("x")
y = theano.tensor.vector("y")
out = x*(y/x) + 0

In [ ]:
theano.printing.debugprint(out)

In [ ]:
fgraph = theano.function([x,y], [out])
theano.printing.debugprint(fgraph)

In [ ]:
fgraph([1],[3])

In [ ]:
theano.printing.pydotprint(out, outfile="img/symbolic_graph_unopt.png", var_with_name_simple=False, high_contrast=False, with_ids=True)
theano.printing.pydotprint(fgraph, outfile="img/symbolic_graph_opt.png", var_with_name_simple=False, high_contrast=False, with_ids=True)

In [ ]:
with pm.Model() as model_normal:
    x = pm.Normal("x", 0., 1.)
    
theano.printing.debugprint(model_normal.logpt)

## Transform Case Study

## PPL from scratch

In [ ]:
# Draw 2 samples from a Normal(1., 2.) distribution
x = stats.norm.rvs(loc=1.0, scale=2.0, size=2, random_state=1234)
# Evaluate the log probability of the samples 
logp = stats.norm.logpdf(x, loc=1.0, scale=2.0)

print(x, logp)

In [ ]:
random_variable_x = stats.norm(loc=1.0, scale=2.0)

x = random_variable_x.rvs(size=2, random_state=1234)
logp = random_variable_x.logpdf(x)

print(x, logp)

In [ ]:
x = stats.norm(loc=1.0, scale=2.0)
y = stats.norm(loc=x, scale=1.)
# y.rvs()

In [ ]:
class RandomVariable:
    def __init__(self, distribution):
        self.distribution = distribution

    def __array__(self):
        return np.asarray(self.distribution.rvs())

x = RandomVariable(stats.norm(loc=1.0, scale=2.0))
z = RandomVariable(stats.halfnorm(loc=0., scale=1.))
y = RandomVariable(stats.norm(loc=x, scale=z))

for i in range(5):
    print(np.asarray(y))

In [ ]:
class RandomVariable:
    def __init__(self, distribution, **kwargs):
        self.distribution = distribution
        self.kwargs = kwargs
    
    def __array__(self):
        return np.asarray(self.distribution(**self.kwargs).rvs())

x = RandomVariable(stats.norm, loc=1.0, scale=2.0)
y = RandomVariable(stats.norm, loc=x, scale=1.)
np.asarray(y)

In [ ]:
class RandomVariable:
    def __init__(self, distribution, value=None):
        self.distribution = distribution
        self.set_value(value)
    
    def __repr__(self):
        return f"{self.__class__.__name__}(value={self.__array__()})"

    def __array__(self, dtype=None):
        if self.value is None:
            return np.asarray(self.distribution.rvs(), dtype=dtype)
        return self.value

    def set_value(self, value=None):
        self.value = value
    
    def log_prob(self, value=None):
        if value is not None:
            self.set_value(value)
        return self.distribution.logpdf(np.array(self))

x = RandomVariable(stats.norm(loc=1.0, scale=2.0))
z = RandomVariable(stats.halfnorm(loc=0., scale=1.))
y = RandomVariable(stats.norm(loc=x, scale=z))

In [ ]:
for i in range(3):
    print(y)

print(f"  Set x=5 and z=0.1")
x.set_value(np.asarray(5))
z.set_value(np.asarray(0.05))
for i in range(3):
    print(y)

print(f"  Reset z")
z.set_value(None)
for i in range(3):
    print(y)

In [ ]:
x.set_value(np.array(5.))
z.set_value(np.array(3.))
x.log_prob() + z.log_prob() + y.log_prob(np.array(5.))

In [ ]:
# Observed y = 5.
y.set_value(np.array(5.))

posterior_density = lambda xval, zval: x.log_prob(xval) + z.log_prob(zval) + y.log_prob()
posterior_density(np.array(0.), np.array(1.))

In [ ]:
def log_prob(xval, zval, yval=5):
    x_dist = stats.norm(loc=1.0, scale=2.0)
    z_dist = stats.halfnorm(loc=0., scale=1.)
    y_dist = stats.norm(loc=xval, scale=zval)
    return x_dist.logpdf(xval) + z_dist.logpdf(zval) + y_dist.logpdf(yval)

log_prob(0, 1)

In [ ]:
xval, zval = 10, 5
posterior_density(np.array(xval), np.array(zval)), log_prob(xval, zval)

In [ ]:
def prior_sample():
    x = stats.norm(loc=1.0, scale=2.0).rvs()
    z = stats.halfnorm(loc=0., scale=1.).rvs()
    y = stats.norm(loc=x, scale=z).rvs()
    return x, z, y

prior_sample()

### Shape handling

In [ ]:
def prior_sample(size):
    x = stats.norm(loc=1.0, scale=2.0).rvs(size=size)
    z = stats.halfnorm(loc=0., scale=1.).rvs(size=size)
    y = stats.norm(loc=x, scale=z).rvs()
    return x, z, y

print([x.shape for x in prior_sample(size=(2))])
print([x.shape for x in prior_sample(size=(2, 3, 5))])

In [ ]:
n_row, n_feature = 1000, 5
X = np.random.randn(n_row, n_feature)

def lm_prior_sample0():
    intercept = stats.norm(loc=0, scale=10.0).rvs()
    beta = stats.norm(loc=np.zeros(n_feature), scale=10.0).rvs()
    sigma = stats.halfnorm(loc=0., scale=1.).rvs()
    y_hat = X @ beta + intercept
    y = stats.norm(loc=y_hat, scale=sigma).rvs()
    return intercept, beta, sigma, y

print([x.shape for x in lm_prior_sample0()])

def lm_prior_sample(size=10):
    if isinstance(size, int):
        size = (size,)
    else:
        size = tuple(size)
    intercept = stats.norm(loc=0, scale=10.0).rvs(size=size)
    beta = stats.norm(loc=np.zeros(n_feature), scale=10.0).rvs(
        size=size + (n_feature,))
    sigma = stats.halfnorm(loc=0., scale=1.).rvs(size=size)
    y_hat = np.squeeze(X @ beta[..., None]) + intercept[..., None]
#     y_hat = np.einsum('ij,...j->...i', X, beta) + intercept[..., None]
    y = stats.norm(loc=y_hat, scale=sigma[..., None]).rvs()
    return intercept, beta, sigma, y

print([x.shape for x in lm_prior_sample(size=())])
print([x.shape for x in lm_prior_sample(size=10)])
print([x.shape for x in lm_prior_sample(size=[10, 3])])

In [ ]:
def lm_prior_sample2(size=10):
    intercept = stats.norm(loc=0, scale=10.0).rvs(size=size)
    beta = stats.multivariate_normal(
        mean=np.zeros(n_feature), cov=10.0).rvs(size=size)
    sigma = stats.halfnorm(loc=0., scale=1.).rvs(size=size)
    y_hat = np.einsum('ij,...j->...i', X, beta) + intercept[..., None]
    y = stats.norm(loc=y_hat, scale=sigma[..., None]).rvs()
    return intercept, beta, sigma, y

print([x.shape for x in lm_prior_sample2(size=())])
print([x.shape for x in lm_prior_sample2(size=10)])
print([x.shape for x in lm_prior_sample2(size=(10, 3))])

In [ ]:
import tensorflow_probability as tfp

tfd = tfp.distributions
tfb = tfp.bijectors

lognormal0 = tfd.LogNormal(0., 1.)
lognormal1 = tfd.TransformedDistribution(tfd.Normal(0., 1.), tfb.Exp())
x = lognormal0.sample(100)

np.testing.assert_array_equal(lognormal0.log_prob(x), lognormal1.log_prob(x))

In [ ]:
# tfb.MaskedAutoregressiveFlow
# tfb.RealNVP

In [ ]:
import tensorflow as tf
X = tf.constant(X, dtype=tf.float32)

In [ ]:
jd = tfd.JointDistributionSequential([
    tfd.Normal(0, 10),
    tfd.Sample(tfd.Normal(0, 10), n_feature),
    tfd.HalfNormal(1),
    lambda sigma, beta, intercept: tfd.Independent(
        tfd.Normal(
            loc=tf.einsum('ij,...j->...i', X, beta) + intercept[..., None],
            scale=sigma[..., None]
        ),
        reinterpreted_batch_ndims=1,
        name='y'
    )
])

print(jd)

n_sample = [3, 2]
for log_prob_part in jd.log_prob_parts(jd.sample(n_sample)):
    assert log_prob_part.shape == n_sample

In [ ]:
# Incorrect if you forgot to wrap tfd.Independent

# jd = tfd.JointDistributionSequential([
#     tfd.Normal(0, 10),
#     tfd.Sample(tfd.Normal(0, 10), n_feature),
#     tfd.HalfNormal(1),
#     lambda sigma, beta, intercept: tfd.Normal(
#             loc=tf.einsum('ij,...j->...i', X, beta) + intercept[..., None],
#             scale=sigma[..., None],
#             name='y'
#     )
# ])

# jd

In [ ]:
# Using tfd.JointDistributionSequentialAutoBatched

# jd = tfd.JointDistributionSequentialAutoBatched([
#     tfd.Normal(0, 10),
#     tfd.Sample(tfd.Normal(0, 10), n_feature),
#     tfd.HalfNormal(1),
#     lambda sigma, beta, intercept: tfd.Normal(
#         loc=X @ beta[..., None] + intercept,
#         scale=sigma,
#         name='y')
# ])

# jd

In [ ]:
dist, value = jd.sample_distributions(3)
dist

In [ ]:
jd.log_prob_parts(jd.sample(3))

### Effect handling

In [ ]:
import numpy as np

import jax
import numpyro
from tensorflow_probability.substrates import jax as tfp_jax

tfp_dist = tfp_jax.distributions
numpyro_dist = numpyro.distributions

init_key = jax.random.PRNGKey(52346)

In [ ]:
def tfp_model():
    x = yield tfp_dist.Normal(loc=1.0, scale=2.0, name="x")
    z = yield tfp_dist.HalfNormal(scale=1., name="z")
    y = yield tfp_dist.Normal(loc=x, scale=z, name="y")
    
def numpyro_model():
    x = numpyro.sample("x", numpyro_dist.Normal(loc=1.0, scale=2.0))
    z = numpyro.sample("z", numpyro_dist.HalfNormal(scale=1.0))
    y = numpyro.sample("y", numpyro_dist.Normal(loc=x, scale=z))

In [ ]:
try:
    print(tfp_model())
except:
    pass

In [ ]:
try:
    print(numpyro_model())
except:
    pass

In [ ]:
init_key, sample_key = jax.random.split(init_key, 2)
tfp_dist.Normal(0., 1.).sample(seed=sample_key), numpyro_dist.Normal(0., 1.).sample(key=sample_key)

In [ ]:
sample_key = jax.random.PRNGKey(52346)

# Draw samples
jd = tfp_dist.JointDistributionCoroutineAutoBatched(tfp_model)
tfp_sample = jd.sample(1, seed=sample_key)

predictive = numpyro.infer.Predictive(numpyro_model, num_samples=1)
numpyro_sample = predictive(sample_key)

# Evaluate log prob
log_likelihood_tfp = jd.log_prob(tfp_sample)
log_likelihood_numpyro = numpyro.infer.util.log_density(
    numpyro_model, [], {}, params=tfp_sample._asdict())

# Same log prob
np.testing.assert_allclose(log_likelihood_tfp, log_likelihood_numpyro[0])

In [ ]:
log_likelihood_tfp, tfp_sample, log_likelihood_numpyro

In [ ]:
# Condition z to .01 in TFP
jd.sample(z=.01, seed=sample_key)

# Condition z to .01 in NumPyro
predictive = numpyro.infer.Predictive(
    numpyro_model, num_samples=1, params={'z': np.asarray(.01)})
predictive(sample_key)

In [ ]:
# Conditioned z to .01 in TFP and construct conditional distributions
dist, value = jd.sample_distributions(z=.01, seed=sample_key)
assert dist.y.loc == value.x
assert dist.y.scale == value.z

# Conditioned z to .01 in NumPyro and construct conditional distributions
model = numpyro.handlers.substitute(numpyro_model, data={'z': .01})
with numpyro.handlers.seed(rng_seed=sample_key):
    # Under the seed context, the default behavior of a NumPyro model is the
    # same as in Pyro: drawing prior sample.
    model_trace = numpyro.handlers.trace(numpyro_model).get_trace()
assert model_trace['y']['fn'].loc == model_trace['x']['value']
assert model_trace['y']['fn'].scale == model_trace['z']['value']

In [ ]:
def numpyro_model_with_input(a):
    x = numpyro.sample("x", numpyro_dist.Normal(loc=1.0, scale=2.0))
    z = numpyro.sample("z", numpyro_dist.HalfNormal(scale=1.0))
    y = numpyro.sample("y", numpyro_dist.Normal(loc=x, scale=z))
    k = numpyro.sample("k", numpyro_dist.Poisson(rate=y))
    return x, z, y, k, z + a

with numpyro.handlers.seed(rng_seed=sample_key):
    # Under the seed context, the default behavior of a NumPyro model is the
    # same as in Pyro: drawing prior sample.
    print(numpyro_model_with_input(a=5.))
    conditioned_model = numpyro.handlers.condition(numpyro_model_with_input, {'z': .01})
    print(conditioned_model(a=5.))
    
conditioned_model_tfp = jd.experimental_pin(z=.01)
conditioned_model_tfp.sample_unpinned(seed=sample_key)

In [ ]:
model = numpyro.handlers.substitute(numpyro_model_with_input,
                                    data={'z': .01, 'x': 3., 'y': 3.1, 'k': 5})
model_trace = numpyro.handlers.trace(model).get_trace(a=5.)

log_joint = jax.numpy.array(0.)
for site in model_trace.values():
    log_prob = site['fn'].log_prob(site['value'])
    log_prob = jax.numpy.sum(log_prob)
    log_joint = log_joint + log_prob